In [ ]:
from IPython.display import display, clear_output, HTML
import base64
#import IPython.display as ipyds
import ipyvuetify as v
import pandas as pd
#import numpy as np
import ipywidgets as widgets
import time as t
#import matplotlib.pylab as plt
from run.hydration import bouges_composition
from run.hydration import parrot_killoh
from run.hydration import run_hydration
from run.hydration import plot_bars
from run.hydration import phase_plot
from run.hydration import to_phase_first_dict
#import plotly.express as px
import plotly.graph_objects as go
import plotly.express as px
#from ipysheet import from_dataframe
import os
import io
import subprocess
import signal

In [ ]:
#import matplotlib as mpl
#mpl.rcParams['figure.figsize']=[15,15]

In [ ]:
def make_dropdown(label_, dic_):
    return v.Select(label=label_, v_model = "",
    items=dic_[label_], style_='width: 250px; opacity: 1')

#parameters for parrot and killoh model source: lothenbach et al. 2008
K1 = {"C3S":1.5,"C2S":0.5,"C3A":1.,"C4AF":0.37}
N1 = {"C3S":0.7,"C2S":1.0,"C3A":0.85,"C4AF":0.7}
K2 = {"C3S":0.05,"C2S":0.02,"C3A":0.04,"C4AF":0.015}
K3 = {"C3S":1.1,"C2S":0.7,"C3A":1.0,"C4AF":0.4}
N3 = {"C3S":3.3,"C2S":5.0,"C3A":3.2,"C4AF":3.7}
H  = {"C3S":1.8,"C2S":1.35,"C3A":1.6,"C4AF":1.45}
Ea = {"C3S":41570,"C2S":20785,"C3A":54040,"C4AF":34087} #from barbara's excel sheets in paper only 2 leading digits are given
T0 = 20#c
ref_fineness = 385

def parrot_killoh():
    DoH = {}
    for phase in ["C3S","C2S","C3A","C4AF"]:
        ivp_out = solve_ivp(overall_rate, [0,np.max(output_times)], [1e-15],t_eval=output_times
                           ,args=(K1[phase],N1[phase],K2[phase],K3[phase],N3[phase],
                           H[phase],wc,RH,fineness,ref_fineness,T0,Ea[phase]))
        DoH[phase] = np.squeeze(ivp_out.y[0])
    return DoH

In [ ]:
# models selection
#GeoChemProcesses = {'Geochemical Processes':  ['Cement Hydration', 'Test ORCHESTRA', 'Cement Carbonation', 'Metal Corrosion', 'Organics Degradation']}
GeoChemProcesses = {'Geochemical Processes':  ['Cement Hydration', 'Cement Carbonation', 'Metal Corrosion', 'Organics Degradation']}
dropdownGeoChemProcesses = make_dropdown('Geochemical Processes', GeoChemProcesses)
MechProcesses = {'Mechanical Processes': ['Porosity/Permeability Evolution', 'Strain Field Evolution', 'Other...']}
dropdownMechProcesses = make_dropdown('Mechanical Processes', MechProcesses)
SurogateProcesses = {'Surogate modeling': ['Train Surogate Models', 'Test Surogate Models']}
dropdownSurogateProcesses = make_dropdown('Surogate modeling', SurogateProcesses)
#dropdownProcesses

In [ ]:
container_dropdown = v.Container(_metadata={'mount_id': 'content-nav'}, class_='px-1 mx-0', 
                                 children=[dropdownGeoChemProcesses, dropdownMechProcesses, dropdownSurogateProcesses])

In [ ]:
#input list
'''
CaO=64.18
SiO2=21.01
Al2O3=4.63
Fe2O3=2.6
SO3=2.78
MgO=1.82
CSH2=2.2 #gypsum 
'''
form_input_cement = v.Form(label='input',  class_='px-0 mx-5', children=[v.Container(children=[
    v.Row(children=[
        v.TextField(v_model='oxide composition in weight %', filled = True, disabled=True,
                    label="Cement composition", value = 'info')
    ]),
    v.Row(children=[
        v.TextField(label="CaO", value = '64.18', v_model = '64.18')
    ]),
    v.Row(children=[
        v.TextField(label="SiO2", value = '21.01', v_model = '21.01')
    ]),
    v.Row(children=[
        v.TextField(label="Al2O3", value = '4.63', v_model = '4.63')
    ]),
    v.Row(children=[
        v.TextField(label="Fe2O3", value = '2.6', v_model = '2.6')
    ]),
    v.Row(children=[
        v.TextField(label="SO3", value = '2.78', v_model = '2.78')
    ]),
    v.Row(children=[
        v.TextField(label="MgO", value = '1.82', v_model = '1.82')
    ]),
    v.Row(children=[
        v.TextField(label="Gypsum", value = '2.2', v_model = '2.2')
    ])
])])

# input other
form_input_other = v.Form(label='input-other', class_='px-0 mx-10', children=[v.Container(children=[
    v.Row(children=[
        v.TextField(v_model='input for the model mix composition', filled = True, disabled=True,
                    label="Other input", value = 'info')
    ]),
    v.Row(children=[
        v.TextField(label="water/cement", value = '0.5', v_model = '0.5')
    ]),
    v.Row(children=[
        v.TextField(label="Relative-Humidity", value = '1', v_model = '1')
    ]),
        v.Row(children=[
        v.TextField(label="Temperature C", value = '45', v_model = '45', disabled=True)
    ]),
        v.Row(children=[
        v.TextField(label="Fineness", value = '385', v_model = '385', disabled=True)
    ])
])])

In [ ]:
clink_phases = {}
c3s = '64.6'
clink_phases["C3S"]=c3s
c2s='9.3'
clink_phases["C2S"]=c2s
c3a= '7.4'
clink_phases["C3A"]=c3a
c4af='7.8'
clink_phases["C4AF"]=c4af

In [ ]:
input_carbonation_boundary = {
'depth':'0',
'A':'1e-4',
'dx':'0',
'totvolume':'1',
'porosity':'1',
'pH':'5',
'SiO2.liter':'1e-9',
'Ca+2.liter':'1e-9',
'H+.liter':'-2e-9',
'O2.liter':'1e-6',
'X':'0',
'Y':'0',
'CO2[g].liter':'1e-2'
}

input_carbonation_grid = {
'depth':'1',
'A':'1e-4',
'dx':'0.005',
'totvolume':'0.001',
'porosity':'0.5',
'pH':'12.47',
'SiO2.liter':'3.146',
'Ca+2.liter':'11.98',
'H+.liter':'-2e-9',
'O2.liter':'1e-6',
'X':'0',
'Y':'0',
'CO2[g].liter':'1e-8'
}


In [ ]:
def make_row_input(label_, value_):
    return v.Row(children=[
        v.TextField(label=label_, value = '' , v_model = value_)
    ])

In [ ]:
def make_form_input(header_m, header_l, dic_lable_value):
    x = [v.Row(children=[
        v.TextField(v_model=header_m, filled = True, disabled=True,
                    label=header_l, value = 'info')
    ])]
    y = [make_row_input(label_,value_) for label_, value_ in dic_lable_value.items()]
    #should be the childeren of row
    return v.Form(label='input',  class_='px-0 mx-10', children=x+y)


In [ ]:
form_input_clinker = v.Container(children=[
    make_form_input('clinker composition in weight %', "Cement composition", clink_phases )])
#form_input_clinker

In [ ]:
form_input_carbonation_boundary = v.Container(children=[
    make_form_input('composition of cell 0', "Boundary condition", input_carbonation_boundary )])
#form_input_carbonation_boundary
form_input_carbonation_grid = v.Container(children=[
    make_form_input('of cells 1-400 of 2d grid', "Define contents", input_carbonation_grid )])
#form_input_carbonation_grid

In [ ]:
def from_inputs_to_dic(inputs):
    dic = {}
    for i in inputs:
        if i.children[0].value != 'info':
            dic[i.children[0].label]=float(i.children[0].v_model)
    return dic

In [ ]:
output_plot1 = widgets.Output(layout = { 'min_width': '800px', 'height': '100%'})
output_plot_carbonation = widgets.Output(layout = { 'min_width': '800px', 'height': '100%'})
#output_plot_carbonation2 = widgets.Output(layout = { 'min_width': '800px', 'height': '100%'})
#output_plot2 = widgets.Output(layout = { 'min_width': '800px', 'height': '100%'})

layout = {
            'width': '100%',
            'height': '500px',
            'border': '1px solid black'
        }

In [ ]:
table_container = v.Container(children=[ ], style_='width: 890px;') #fluid_style="height: 120vh;"
orchestra_container_output = v.Container(children = [], style_="min-width: 900px;")
cem_carbonation_table_container = v.Container(children = [], style_="min-width: 900px;")
mm = widgets.HTMLMath(
    value=r"Some math and <i>HTML</i>: \(x^2\) and $$\frac{x+1}{x-1}$$",
    placeholder='Some HTML',
    description='Some HTML',
)
cem_carbonation_description_container = v.Container(children = [v.TextField( disabled=True, value = 'If pH<12, Overall Integrity = 1-[(12-pH)/7)]')], style_="min-width: 900px;")

In [ ]:
def create_csv_download_link(filename):
    df = pd.read_csv(filename)
    title = "download the " + filename + " file"
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" text-decoration: none" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    #html = '<v-btn class="ma-2" outlined href="data:text/csv;base64,{payload}" target="_blank" download > Download PDF </v-btn>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

In [ ]:
# file download
dld_hyd_btn = v.Tooltip(bottom=True, v_slots=[{
                                        'name': 'activator',
                                        'variable': 'tooltip',
                                        'children': v.Btn(v_on='tooltip.on',
                                                          class_="ma-2" ,
                                                          href="../../results.csv",
                                                          download=True,
                                                          target="_blank",
                                                          color='blue-grey', children=['Download Results']),
                                        }], children=['download the results file'])


In [ ]:
# kill process
kill_btn = v.Tooltip(bottom=True, v_slots=[{
                                        'name': 'activator',
                                        'variable': 'tooltip',
                                        'children': v.Btn(v_on='tooltip.on', class_="ma-2" , color='red', children=['Stop Process']),
                                        }], children=['Results for the run until STOP will be shown'])

In [ ]:
def make_tabs(inputs, outputs):
    return v.Tabs( class_='mx-4', children=[
           v.Tab(children=['Process Input']),
           v.Tab(children=['Process Results']),
           v.TabItem(children=[v.Layout(row =True, wrap=True, align_top=True, 
                        children=[v.Flex(xs12=True, lg6=True, xl4=False, children=[input_]) for input_ in inputs ]) ]),
           v.TabItem(children=[v.Layout(column=True, wrap=True, align_center=True, 
                        children=[v.Flex(xs12=True, lg6=True, xl4=True, class_='px-2 mx-0', children=[output_]) for output_ in outputs ]) ])
           ])

In [ ]:
orchestra_input_files = widgets.FileUpload(multiple=True, lable='xx')#FileInput()

In [ ]:
orchestra_container_input = v.Container(children = [
    v.Row(class_='mt-2 ml-2',children=[v.Tooltip(bottom=True, v_slots=[{'name': 'activator',
                                 'variable': 'tooltip',
                                 'children': v.Chip(v_on='tooltip.on', children=['Upload user edited ORCHESTRA files']),
                                }], children=['inp: composer, chemistry, ...; dat: input; xml: concert.xml, etc. Hold Ctrl to select multiple'])]),
    v.Row(class_='mt-2 ml-2',children=[orchestra_input_files])

])

In [ ]:
tabs_hydration = make_tabs([form_input_cement, form_input_other], [output_plot1, table_container])
tabs_hydration = make_tabs([form_input_clinker, form_input_other], [output_plot1, table_container])
tabs_carbonation_orchestra = make_tabs([form_input_carbonation_boundary, form_input_carbonation_grid], 
                                       [cem_carbonation_description_container, output_plot_carbonation, cem_carbonation_table_container])
tabs_carbonation = make_tabs([], [])
#tabs_orchestra = make_tabs([orchestra_container_input], [orchestra_container_output])
#tabs

with output_plot_carbonation2:
    display(HTML("""
<script type="text/javascript" async
  src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.7/MathJax.js?config=config=TeX-AMS-MML_SVG">
</script>
"""))
    display(widgets.Text(description="$\sigma$") )

In [ ]:
def make_card_process(title_, subtitle_, tabs_):
    return v.Card( children = [v.CardTitle(children=title_),
                               v.CardSubtitle(children=subtitle_), 
                               v.Layout( row=True, wrap=True, align_center=True, children=[
                                        v.Flex(xs12=True, lg6=True, xl4=True, children=[
                                            tabs_]) ]),
                               v.CardActions(class_='pl-4 ml-4', children=[
                                   v.Tooltip(bottom=True, v_slots=[{
                                        'name': 'activator',
                                        'variable': 'tooltip',
                                        'children': v.Btn(v_on='tooltip.on', color='primary', children=['Run Process']),
                                        }], children=['Run then switch to PROCESS RESULTS'])
]) ], 
                    elevation="2", outlined=True, loading=False)

In [ ]:
# Other process card
card_process_trainNN = v.Card(children = [
    v.CardTitle(children="Train Cement Hydration surogate model"),
    v.CardSubtitle(children="here the user could setup and train a cement hydration surogate model")
], elevation="2", outlined=True)

card_process_testNN = v.Card(children = [
    v.CardTitle(children="Test trained surogate models"),
    v.CardSubtitle(children="Here the user could test/run surogate models for different eveolution processes")
], elevation="2", outlined=True)

card_process_mechanics = v.Card(children = [
    v.CardTitle(children="Mechanical properties"),
    v.CardSubtitle(children="here the user could run a process on the mechanical properties of the waste pacakge that uses some output that from previous (chemical) processes")
], elevation="2", outlined=True)

# Cement hydration process card
card_process_cem_hydration = make_card_process('Cement hydration', 
                                           'OPC cement paste hydration process using Parrot and Killoh hydration model, GEMS and CEMDATA18.',
                                           tabs_hydration)
#card_process_test_orchestra = make_card_process('Test ORCHESTRA', 
#                                           'This is a test for runing ORCHESTRA from jupyter',
#                                           tabs_orchestra)

# carbonation
card_process_cem_crabonation_orchestra = make_card_process('Cement Carbonation', 
                                           'Cement carbonation by CO2 difusion model using ORCHESTRA ...',
                                           tabs_carbonation_orchestra)

card_process_cem_crabonation_orchestra.children[3].children.append(kill_btn)


card_process_met_corrosion = make_card_process('Metal Corrosion', 
                                           'Model to run Metal Corrosion ...',
                                           tabs_carbonation)
card_process_org_degradation = make_card_process('Organics Degradation', 
                                           'Model to run Organics Degradation ...',
                                           tabs_carbonation)

# main card added to the web-widget
#card_main = v.Card(_metadata={'mount_id': 'content-main'}, children = [])
container_main = v.Container(_metadata={'mount_id': 'content-main'}, fluid=True, class_='pl-4 ml-4', children=[])

In [ ]:
def check(list1, val):
    for e in list1:
        if e > val:
            return True
    return False

In [ ]:
import pandas as pd
import traitlets
import ipyvuetify as v
import json

class PandasDataFrame(v.VuetifyTemplate):
    """
    Vuetify DataTable rendering of a pandas DataFrame

    Args:
        data (DataFrame) - the data to render
        title (str) - optional title
    """

    headers = traitlets.List([]).tag(sync=True, allow_null=True)
    items = traitlets.List([]).tag(sync=True, allow_null=True)
    search = traitlets.Unicode('').tag(sync=True)
    title = traitlets.Unicode('DataFrame').tag(sync=True)
    index_col = traitlets.Unicode('').tag(sync=True)
    template = traitlets.Unicode('''
        <template>
          <v-card>
            <v-card-title>
              <span class="title font-weight-bold">{{ title }}</span>
              <v-spacer></v-spacer>
                <v-text-field
                    v-model="search"
                    append-icon="search"
                    label="Search ..."
                    single-line
                    hide-details
                ></v-text-field>
            </v-card-title>
            <v-data-table
                dense
                :headers="headers"
                :items="items"
                :search="search"
                :item-key="index_col"
                :rows-per-page-items="[25, 50, 250, 500]"
            >
                <template v-slot:no-data>
                  <v-alert :value="true" color="error" icon="warning">
                    Sorry, nothing to display here :(
                  </v-alert>
                </template>
                <template v-slot:no-results>
                    <v-alert :value="true" color="error" icon="warning">
                      Your search for "{{ search }}" found no results.
                    </v-alert>
                </template>
                <template v-slot:items="rows">
                  <td v-for="(element, label, index) in rows.item"
                      @click=cell_click(element)
                      >
                    {{ element }}
                  </td>
                </template>
            </v-data-table>
          </v-card>
        </template>
        ''').tag(sync=True)

    def __init__(self, *args,
                 data=pd.DataFrame(),
                 title=None,
                 **kwargs):
        super().__init__(*args, **kwargs)
        data = data.reset_index()
        self.index_col = data.columns[0]
        headers = [{
              "text": col,
              "value": col
            } for col in data.columns]
        headers[0].update({'align': 'left', 'sortable': True})
        self.headers = headers
        self.items = json.loads(
            data.to_json(orient='records'))
        if title is not None:
            self.title = title

In [ ]:
# Run simulation click
count = 0

def on_click_geoc_hydr(widget, event, data):
    global count
    #amount = form_input_cement.children[0].children[count].children[0].v_model
    #card_process_hydration.children[3].children[0].children=[f'Click me {amount}']
    card_process_cem_hydration.loading = True
    old_btn = card_process_cem_hydration.children[3].children[0].v_slots[0]['children'].children
    progress = v.ProgressCircular(indeterminate = True, color="white")
    card_process_cem_hydration.children[3].children[0].v_slots[0]['children'].children=[progress]
    #t.sleep(1)
    #count += 1
    
    CaO=float(form_input_cement.children[0].children[1].children[0].v_model)
    SiO2=float(form_input_cement.children[0].children[2].children[0].v_model)
    Al2O3=float(form_input_cement.children[0].children[3].children[0].v_model)
    Fe2O3=float(form_input_cement.children[0].children[4].children[0].v_model)
    SO3=float(form_input_cement.children[0].children[5].children[0].v_model)
    MgO=float(form_input_cement.children[0].children[6].children[0].v_model)
    CSH2=float(form_input_cement.children[0].children[7].children[0].v_model)
    
    wc = float(form_input_other.children[0].children[1].children[0].v_model)
    RH = float(form_input_other.children[0].children[2].children[0].v_model)
    T = float(form_input_other.children[0].children[3].children[0].v_model)
    fineness = float(form_input_other.children[0].children[4].children[0].v_model)
    
    #clinker_phases = bouges_composition(CaO,SiO2,Al2O3,Fe2O3,SO3)
    clinker_phases = from_inputs_to_dic(form_input_clinker.children[0].children)
    
    pk = parrot_killoh(wc, RH, T, fineness)
    
    try:
        vol_frac, mass_frac, density = run_hydration(clinker_phases, wc, CSH2, T, pk)
    except Exception as e:
        print(e)
    
    with output_plot1:         
        clear_output(wait=True)
        gems_vol_frac_phase_first = to_phase_first_dict(vol_frac)
        gems_masses_phase_first = to_phase_first_dict(mass_frac)
        #p2 = phase_plot(gems_vol_frac_phase_first)
        #p2.show()
        
        x= [0.,0.1,1,3,7,28,30,60,90,365, 730,1000,2000]
        #['aq_gen', 'C3(AF)S0_84H', 'CSHQ', 'Belite']
        
        size = 0

        fig = go.Figure()
        vol_frac_simple = {}
        for g in gems_vol_frac_phase_first:
            if  check(gems_vol_frac_phase_first[g], 1e-3):#np.count_nonzero(gems_vol_frac_phase_first[g]) > 0:
                size=size+1
                vol_frac_simple[g] = gems_vol_frac_phase_first[g]
                fig.add_trace(go.Scatter(
                    x=x, y=gems_vol_frac_phase_first[g],
                    hoverinfo='x+y',
                    name=g,
                    mode='lines',
                    line=dict(width=0.5), #color='rgb(131, 90, 241)'),
                    stackgroup='one', # define stack group
                    hovertemplate='%{y}'
                ))
                
        for g in gems_masses_phase_first:
            if  check(gems_masses_phase_first[g], 1e-3):#np.count_nonzero(gems_vol_frac_phase_first[g]) > 0:
                size=size+1
                fig.add_trace(go.Scatter(
                    x=x, y=gems_masses_phase_first[g],
                    hoverinfo='x+y',
                    name=g,
                    mode='lines',
                    line=dict(width=0.5), #color='rgb(131, 90, 241)'),
                    stackgroup='two', # define stack group
                    hovertemplate='%{y}',
                    visible=False
                ))  


        fig.update_xaxes(type="log")
        fig.update_layout(  yaxis_range=(0, 1), 
                            title = "volume fraction",
                            xaxis_title="Time [days]",
                            yaxis_title="Volume fraction [m^3/m^3 of initial volume]",
                            legend_title="Phases",
                            hovermode='x',     
                              updatemenus=[
                dict(
                buttons=list([
                    dict(
                        #args=["type", "volfrac"],q.index(v) if v in q else 99999 for v in vm
                        args = [{"visible": [ True if x<size/2 else False for x in range(size) ]}, 
                                {"xaxis.type": "log", "yaxis.range" : [0, 1], 
                                 "yaxis.title": "Volume fraction [m^3/m^3 of initial volume]", 
                                 "xaxis.title": "Time [days]", "title": "volume fraction"}],
                        label="Volume Fraction",
                        method="update"
                        ),
                    dict(
                        #args=["type", "masfrac"],
                        args = [{"visible": [ False if x<size/2 else True for x in range(size) ]},
                                {"xaxis.type": "log", "yaxis.range" : [0, 0.1435], 
                                 "yaxis.title": "mass [g/100 g of cement]", 
                                 "xaxis.title": "Time [days]", "title": "mass fraction"}],
                        label="Mass Fraction",
                        method="update"
                    )
                ]),
                direction="down",
                pad={"r": 10, "t": 10},
                showactive=True,
                x=1.0,
                xanchor="left",
                y=1.15,
                yanchor="top"
                )
        ])
        
        pd_vol_frac = pd.DataFrame.from_dict(vol_frac_simple)
        
        pd_vol_frac.to_csv('results.csv')
        
        fig.show()
        
        iris = pd.read_csv('results.csv')
        test = PandasDataFrame(data = iris, title='Volume Fraction')
        table_container.children = [test]
        
        #local_file = FileLink('./results.csv', result_html_prefix="Click here to download: ")
        display(create_csv_download_link('results.csv'))
        
        #ipyds.display(from_dataframe(pd_vol_frac))
        
        #tabs.v_model='tab1'
        
    t.sleep(1)
    
    card_process_cem_hydration.loading = False
    card_process_cem_hydration.children[3].children[0].v_slots[0]['children'].children=old_btn

    #fig1.layout.width = 'auto'
    #fig1.layout.height = 'auto'
    #fig1.layout.min_height = '300px' # so it shows nicely in the notebook
    
card_process_cem_hydration.children[3].children[0].v_slots[0]['children'].on_event('click', on_click_geoc_hydr)

In [ ]:
progress = v.ProgressCircular(indeterminate = True, color="white")

In [ ]:
old_btn = card_process_cem_crabonation_orchestra.children[3].children[0].v_slots[0]['children'].children

In [ ]:
pid = 99999

# Run simulation click
count = 0

def on_click_geoc_orch(widget, event, data):
    global count
    
    card_process_test_orchestra.loading = True
    old_btn = card_process_test_orchestra.children[3].children[0].v_slots[0]['children'].children
    card_process_test_orchestra.children[3].children[0].v_slots[0]['children'].children=[progress]
    
    '''
    do stuff
    '''
    for k,v in orchestra_input_files.value.items():
    #print(k)
    #print('-----')
    #print(v['content'])
        with open('Orchestra/'+k, "wb") as fp:
            fp.write(v['content'])
            
    orchestra_input_files.value.clear()
    orchestra_input_files._counter = 0
    
    os.system('java -cp Orchestra/orchestra2022.jar orchestra2.RunBatch -home Orchestra/ concert.xml')
    
    
    with open('Orchestra/output.dat') as f:
        contents = f.read()
        
    result = contents[contents.find("Var")+ len("Var:"):]
    result2 = result[result.find("Var"):]
    
    data = io.StringIO(result2)
    p = pd.read_csv(data, sep='\t')
    
    table = PandasDataFrame(data = p, title='Orchestra Output')
    
    #text_area = v.Textarea(name='"input-7-1"', filled=True, label='Orchestra Output', rows=20, value=substring_head+substring_data )
    
    
    orchestra_container_output.children = [table]
    
    card_process_test_orchestra.loading = False
    card_process_test_orchestra.children[3].children[0].v_slots[0]['children'].children=old_btn
    
card_process_test_orchestra.children[3].children[0].v_slots[0]['children'].on_event('click', on_click_geoc_orch)

In [ ]:
def on_click_geoc_carbon_orch_kill(widget, event, data):
    with output_plot_carbonation:
        global pid
        os.kill(pid+1, signal.SIGTERM)
        clear_output( wait=True )
        #print (pid)
        with open('orchestra_2d_cement_diffusion/integrity.dat') as f:
            contents = f.read()
        
        result = contents[contents.find("Var"):]
        #result2 = result[result.find("Var"):]
    
        data = io.StringIO(result)
        p = pd.read_csv(data, sep='\t')
    
        p.loc[p['Var: '] == 'Data:', 'Var: '] = 'Cement'
        p1= p.iloc[::5, :]
        p2 = p1.iloc[::5, :]
        p3 = p2.iloc[::5, :]
        p3.iloc[::5, :]
        p4 = p3.drop(columns=['Unnamed: 3'])
        p5 = p4.rename(columns={'Var: ': 'Material'})
    

        fig = px.bar(p5, x="Material", y=" integrity               ", animation_frame=" time_years              ",  range_y=[0,1])
        fig.show()
        
        display(create_csv_download_link('orchestra_2d_cement_diffusion/integrity.dat'))
    
        table = PandasDataFrame(data = p5, title='Integrity Output')
    
    #text_area = v.Textarea(name='"input-7-1"', filled=True, label='Orchestra Output', rows=20, value=substring_head+substring_data )
        cem_carbonation_table_container.children = [table]
        
        card_process_cem_crabonation_orchestra.loading = False
        global old_btn
        card_process_cem_crabonation_orchestra.children[3].children[0].v_slots[0]['children'].children=old_btn




In [ ]:
def write_input_carbonation(input_carbonation_boundary, input_carbonation_grid):
    with open('orchestra_2d_cement_diffusion/parameters_old.txt') as f:
        contents = f.read()
    x = pd.DataFrame([input_carbonation_grid])
    y = pd.DataFrame(index=[0],columns=['Var:'], data=['Default:'])
    df_grid = pd.concat([y,x], axis=1)
    #df_grid.set_index('Var:', inplace=True)
    df_grid_s = df_grid.to_string(index=False) + '\n\n\n'
    x = pd.DataFrame([input_carbonation_boundary])
    y = pd.DataFrame(index=[0],columns=['Var:'], data=['Data:'])
    df_boundary = pd.concat([y,x], axis=1)
    df_boundary_s = df_boundary.to_string(index=False) + '\n\n\n'
    result = contents[:contents.find("Var")]
    result = result + df_boundary_s + df_grid_s + contents[contents.find("Sweep"):]
    with open('orchestra_2d_cement_diffusion/parameters.txt', "w") as fp:
        fp.write(result)


In [ ]:
# Run carbonation click
count = 0

def on_click_geoc_carbon_orch(widget, event, data):
    global count
    
    card_process_cem_crabonation_orchestra.loading = True
    #old_btn = card_process_cem_crabonation_orchestra.children[3].children[0].v_slots[0]['children'].children
    card_process_cem_crabonation_orchestra.children[3].children[0].v_slots[0]['children'].children=[progress]
    
    '''
    do stuff
    '''

    '''
    for k,v in orchestra_input_files.value.items():
    #print(k)
    #print('-----')
    #print(v['content'])
        with open('Orchestra/'+k, "wb") as fp:
            fp.write(v['content'])
            
    orchestra_input_files.value.clear()
    orchestra_input_files._counter = 0
    '''
    #pid = os.fork()
    #if pid == 0: # pid is only equal to 0 in the child process
    #    os.system('java -cp orchestra_2d_cement_diffusion/orchestra2022.jar orchestra2.RunBatch -home orchestra_2d_cement_diffusion/ concert.xml')
    
    #sts = subprocess.Popen('java -cp orchestra_2d_cement_diffusion/orchestra2022.jar orchestra2.RunBatch -home orchestra_2d_cement_diffusion/ concert.xml -nographics', shell=True)
    with open('orchestra_2d_cement_diffusion/integrity_old.dat') as f:
        contents = f.read()
        
    result = contents[contents.find("Var"):]
    #result2 = result[result.find("Var"):]
    
    data = io.StringIO(result)
    p = pd.read_csv(data, sep='\t')
    
    p.loc[p['Var: '] == 'Data:', 'Var: '] = 'Cement'
    p1= p.iloc[::5, :]
    p2 = p1.iloc[::5, :]
    p3 = p2.iloc[::5, :]
    p3.iloc[::5, :]
    p4 = p3.drop(columns=['Unnamed: 3'])
    p5 = p4.rename(columns={'Var: ': 'Material'})
    
    with output_plot_carbonation:         
        fig = px.bar(p5, x="Material", y=" integrity               ", animation_frame=" time_years              ",  range_y=[0,1])
        table = PandasDataFrame(data = p5, title='Integrity Output')
        clear_output(wait=True )
        fig.show()
        
        display(create_csv_download_link('orchestra_2d_cement_diffusion/integrity_old.dat'))
    
    #text_area = v.Textarea(name='"input-7-1"', filled=True, label='Orchestra Output', rows=20, value=substring_head+substring_data )
    cem_carbonation_table_container.children = [table]
    
    t.sleep(3)
    
    input_carbonation_boundary = from_inputs_to_dic(form_input_carbonation_boundary.children[0].children)
    input_carbonation_grid = from_inputs_to_dic(form_input_carbonation_grid.children[0].children)
    
    write_input_carbonation(input_carbonation_boundary, input_carbonation_grid)
    
    sts = subprocess.Popen('java -cp orchestra_2d_cement_diffusion/orchestra2022.jar orchestra2.RunBatch -home orchestra_2d_cement_diffusion/ concert.xml -nographics', shell=True)

    global pid
    pid = sts.pid
    

    pid2 = os.fork()
    if pid2 == 0:
        alive = False
        if sts.poll() is None:
            alive = True
        while alive:
            if sts.poll() is None:
                alive = True
            else:
                global old_btn
                card_process_cem_crabonation_orchestra.loading = False
                card_process_cem_crabonation_orchestra.children[3].children[0].v_slots[0]['children'].children=old_btn
                alive = False
        #if fail_condition:
        #    break

    
    #card_process_cem_crabonation_orchestra.loading = False
    #card_process_cem_crabonation_orchestra.children[3].children[0].v_slots[0]['children'].children=old_btn
    
card_process_cem_crabonation_orchestra.children[3].children[0].v_slots[0]['children'].on_event('click', on_click_geoc_carbon_orch)
card_process_cem_crabonation_orchestra.children[3].children[1].v_slots[0]['children'].on_event('click', on_click_geoc_carbon_orch_kill)

In [ ]:
#vol_frac.head() 'Cement Hydration', 'Mechanical evolution', 'Train Surogate Models', 'Test Surogate Models']

In [ ]:
card_process_cem_hydration

In [ ]:
def dropdownGeoChemProcesses_eventhandler(change):
    if (change.new == "Cement Hydration"):
        container_main.children = [card_process_cem_hydration]
    elif (change.new == "Test ORCHESTRA"):
        container_main.children = [card_process_test_orchestra]
    elif (change.new == "Cement Carbonation"):
        container_main.children = [card_process_cem_crabonation_orchestra]
    elif (change.new == "Metal Corrosion"):
        container_main.children = [card_process_met_corrosion]
    elif (change.new == "Organics Degradation"):
        container_main.children = [card_process_org_degradation]
    #with output:
        #clear_output(wait=True)
        #display(c)
#output = widgets.Output(_metadata={'mount_id': 'content-main'})

In [ ]:
dropdownGeoChemProcesses.observe(dropdownGeoChemProcesses_eventhandler, names='v_model')

In [ ]:
display(dropdownGeoChemProcesses)
#card_main.loading=True
container_main